# Clippy, the friendly literature review bot!

## Download and import necessary packages, API key

In [ ]:
# install PyPI
#%pip install -U google-generativeai
#%pip install PyPDF2
#%pip install tika
#%pip install ast

# importing gemini api
import google.generativeai as genai
import os, glob
import PyPDF2
import re
import pandas as pd
import json
import typing_extensions as typing
from tika import parser
import ast
import datetime

# input api key payload
genai.configure(api_key = YOUR_API_KEY_HERE)

## Instructions and prompt for Clippy and setting location of PDFs to read

In [ ]:
# Prompt  of model
model=genai.GenerativeModel(
    model_name="gemini-1.5-flash", 
    system_instruction="You are a research assistant AI named Clippy. Your job is to help the user look through research papers that will be provided to you.\
                        You are to evaluate whether this paper is relevant to a literature review on infants and explain why in the relevance section of the response.\
                        Date should be written in the form of DD-MM-YYYY\
                        You are to greet the user in a warm and friendly, customer service tone.\
                        Skip preamble and only return what is asked. If you are unsure\
                        Only respond if you are sure of the answer, if not, respond with I do not know") # Change this to whatever you want your prompt to be


class Results(typing.TypedDict): # Choose your output from the abstract that you want Clippy to pull out
    citation: str
    date_of_publication: str
    participant_sample: str
    purpose_of_study: str
    findings: str
    keywords: str
    relevance: str

# Changing directory to papers folder
os.chdir(r'YOUR_FOLDER_HERE') # Change this to the directory of the folder containing your PDF files
## test
print(os.getcwd())

## Running Clippy!
### Output will be stored as xlsx file in PDFs folder

In [ ]:
# file path
file_path=r'C:\Users\greyk\Documents\Python\Clippy research assistant\Papers'

# Create an empty list to append
result=[]

# Implementing the looped function into the Gemini API

for file in os.listdir(file_path):
    if file.endswith('.pdf'):
        pdf_file=open(file,'rb')
        read_pdf=PyPDF2.PdfReader(pdf_file)
        for page_num in range(1):
            page_text=read_pdf.pages[page_num].extract_text().lower()
            page_summary=model.generate_content(page_text, 
                                                generation_config=genai.GenerationConfig(
                                                    response_mime_type='application/json', response_schema=Results),) 
    result.append(pd.json_normalize(json.loads(page_summary.text)))
appended_data=pd.concat(result)

# Viewing the dataframe
appended_data

# Output to csv file
appended_data.to_excel('literature_review.xlsx', index=False)